In [ ]:
# Import
import pandas as pd 

In [ ]:
# Load data
df_customers = pd.read_csv("Olist/olist_customers_dataset.csv")
df_geo = pd.read_csv("Olist/olist_geolocation_dataset.csv")
df_items = pd.read_csv("Olist/olist_order_items_dataset.csv")
df_payments = pd.read_csv("Olist/olist_order_payments_dataset.csv")
df_reviews = pd.read_csv("Olist/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("Olist/olist_orders_dataset.csv")
df_products = pd.read_csv("Olist/olist_products_dataset.csv")
df_sellers = pd.read_csv("Olist/olist_sellers_dataset.csv")
df_cat = pd.read_csv("Olist/product_category_name_translation.csv")


In [ ]:
# Explore data

def basic_information(name):
    print("\n\n Number of  duplicate value ",name.duplicated().sum())
    print("\n\n Missing values\n",name.isnull().sum())
    print("\n\n Column information ")
    name.info()
    print("----------------------------------------------------")
        



# df_customers

In [ ]:
#check
basic_information(df_customers)

In [ ]:
#fix datatype
df_customers['customer_zip_code_prefix'] = df_customers['customer_zip_code_prefix'].astype(str)

# df_orders

In [ ]:
# check
basic_information(df_orders)


In [ ]:
# fix datatype
def to_dt(s):
    return pd.to_datetime(s, errors='coerce')

df_orders['order_purchase_timestamp']=to_dt(df_orders['order_purchase_timestamp'])
df_orders['order_approved_at']=to_dt(df_orders['order_approved_at'])
df_orders['order_delivered_carrier_date']=to_dt(df_orders['order_delivered_carrier_date'])
df_orders['order_delivered_customer_date']=to_dt(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date']=to_dt(df_orders['order_estimated_delivery_date'])

In [ ]:
# new columns
df_orders['order_approved_flag']= df_orders['order_approved_at'].notna()
df_orders['order_shipped_flag']= df_orders['order_delivered_carrier_date'].notna()
df_orders['order_delivered_flag']= df_orders['order_delivered_customer_date'].notna()

In [ ]:
#display
df_orders[['order_status','order_approved_flag','order_shipped_flag','order_delivered_flag']].head()

# df_payments


In [ ]:
#check
basic_information(df_payments)

In [ ]:
# more then one payments
df_payments['order_id'].value_counts().head()

In [ ]:
# number of orders with more then one payment 
df_payments['order_id'].value_counts().gt(1).sum()

In [ ]:
# create aggregated payment table

df_payments_clean=(df_payments.groupby('order_id').agg({'payment_value':'sum',
                                                        'payment_installments':'max',
                                                        'payment_type':lambda x : ','.join(sorted(set(x)))}).reset_index().rename(columns={
                                                            'payment_value':'total_payment_value',
                                                            'payment_installments':'max_installments',
                                                            'payment_type':'payment_types'})
                                                            )
                                                            

In [ ]:
#check
df_payments_clean['payment_types'].value_counts()

In [ ]:
# validation check 
# one row per order
df_payments_clean.shape

In [ ]:
# unique order id
df_payments_clean['order_id'].duplicated().sum()

In [ ]:
#sanity check-total payment for an example order 
exm=df_payments[df_payments['order_id'] == df_payments['order_id'].value_counts().idxmax()]

In [ ]:
#check
len(exm)

In [ ]:
len(df_payments_clean[df_payments_clean['order_id'] == exm.iloc[0]['order_id']])

In [ ]:
#aggregation check
raw_sum=exm['payment_value'].sum()
clean_sum=df_payments_clean[df_payments_clean['order_id'] == exm.iloc[0]['order_id']]['total_payment_value'].iloc[0]
raw_sum , clean_sum

# df_items

In [ ]:
#check
basic_information(df_items)

In [ ]:
# change datatype 
df_items['shipping_limit_date']=pd.to_datetime(df_items['shipping_limit_date'])

In [ ]:
# new columns 
df_items['total_item_value']= df_items['price'] + df_items['freight_value']

In [ ]:
# aggregate by Order_id
order_items_summary = df_items.groupby('order_id').agg({'price':'sum',
                                                        'freight_value':'sum',
                                                        'total_item_value':'sum'}).reset_index().rename(columns={'price':'item_revenue',
                                                                                                        'freight_value':'logistics_cost',
                                                                                                        'total_item_value':'total_item_value'})

# df_products

In [ ]:
#check
basic_information(df_products)

In [ ]:
# replace the missing category
df_products['product_category_name']=df_products['product_category_name'].fillna("unknown_category")

In [ ]:
# marge
df_products=df_products.merge(df_cat,on='product_category_name',how='left')

In [ ]:
# fill numeric missing value with median
for i  in  (df_products.select_dtypes(include='number').columns):
    median_value=df_products[i].median()
    df_products[i]=df_products[i].fillna(median_value)
    print("Fill numeric holes with median for ",i)

In [ ]:
# fill categorical missing value
df_products['product_category_name_english']=df_products['product_category_name_english'].fillna(df_products['product_category_name'])

In [ ]:
# set lowercase
df_products['product_category_name_english']=df_products['product_category_name_english'].str.lower()

In [ ]:
#marge product to items

df_items=df_items.merge(df_products[['product_id', 'product_category_name_english']],on='product_id',how='inner')

# df_sellers

In [ ]:
#check
basic_information(df_sellers)

In [ ]:
# change datatype
df_sellers['seller_zip_code_prefix']=df_sellers['seller_zip_code_prefix'].astype(object)

In [ ]:
#strip whitespace and lowercase
for i in ['seller_id','seller_city','seller_state']:
    if i in df_sellers.columns:
        x=df_sellers[i].str.strip()
        df_sellers[i]=x.str.lower()
        print("Successful")


In [ ]:
# top seller per order

top_seller = ( df_items.groupby('order_id')['seller_id'].agg(lambda x : x.mode().iloc[0] if not x.mode().empty else x.iloc[0]).reset_index(name='top_seller_id'))

In [ ]:
# merge all as fact table
fact = df_orders.copy()
fact = fact.merge(order_items_summary,on='order_id',how='left')
fact = fact.merge(df_payments_clean,on='order_id',how='left')
fact = fact.merge(top_seller,on='order_id',how='left')
fact = fact.merge(df_sellers,left_on='top_seller_id',right_on='seller_id',how='left')

In [ ]:
# Fill missing value
# numerical 
fact[['item_revenue','logistics_cost','total_item_value','total_payment_value','max_installments']]=fact[['item_revenue','logistics_cost','total_item_value','total_payment_value','max_installments']].fillna(0)
# categorical 
fact['payment_types']=fact['payment_types'].fillna('no_payment')
fact[['top_seller_id','seller_id','seller_zip_code_prefix','seller_city','seller_state']]=fact[['top_seller_id','seller_id','seller_zip_code_prefix','seller_city','seller_state']].fillna('unknown')

In [ ]:
#add final kpi
# delivery duration by days
fact['delivery_days'] = (fact['order_delivered_customer_date'] - fact['order_purchase_timestamp']).dt.days

# fulfillment duration by days
fact['fulfillment_days'] = (fact['order_delivered_carrier_date'] - fact['order_purchase_timestamp']).dt.days

# payment gap (+ = overpaid , - = underpaid)
fact['payment_gap'] = fact['total_payment_value'] - fact['total_item_value']              

In [ ]:
#check
fact[['delivery_days','fulfillment_days','payment_gap']].isnull().sum()

In [ ]:
# export file
fact.to_csv("olist_fact_sales.csv")